<center><div style="direction:rtl;font-family:B Lotus, B Nazanin, Tahoma">به نام خدا</div></center>
<h1><center><div style="direction:rtl;font-family:B Lotus, B Nazanin, Tahoma">آموزش یک شبکه عصبی کانولوشنالی از ابتدا</div></center></h1>

In [1]:
import tensorflow as tf
import os
import pathlib
import numpy as np
import matplotlib.pyplot as plt

_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'

path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)

PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

train_dir =  "/root/.keras/datasets/cats_and_dogs_filtered/train"
validation_dir = "/root/.keras/datasets/cats_and_dogs_filtered/validation"

In [2]:
def augment_data(image):
  image = tf.image.resize_with_crop_or_pad(image, 180, 180)
  image = tf.image.random_crop(image, size=[150, 150, 3])
  image = tf.image.random_brightness(image, max_delta=0.5)

  return image

train_dir = pathlib.Path(train_dir)
validation_dir = pathlib.Path(validation_dir)

class_names = np.array([item.name for item in train_dir.glob('*')])

In [3]:
class_names

array(['cats', 'dogs'], dtype='<U4')

In [4]:
train_dataset = tf.data.Dataset.list_files(str(train_dir/'*/*'))

In [5]:
validation_dataset = tf.data.Dataset.list_files(str(validation_dir/'*/*'))

In [6]:
def get_label(filepath):
  parts = tf.strings.split(filepath, os.path.sep)
  return parts[-2] == class_names

In [7]:
def load_image(imagepath):
  image = tf.io.read_file(imagepath)
  image = tf.image.decode_image(image, 3, expand_animations=False)
  image = tf.cast(image, tf.float32)
  return image

In [8]:
def normalize(image):
  image = (image / 255.0)
  return image

In [9]:
def resize(image, height, width):
  image = tf.image.resize(image, (height, width), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
  return image

In [10]:
def load_image_with_label(imagepath):
  label = get_label(imagepath)
  image = load_image(imagepath)
  return image, label

In [11]:
def load_image_train(imagefile):
  image, label = load_image_with_label(imagefile)
  image = augment_data(image)
  image = normalize(image)
  return image, label

In [12]:
def load_image_test(imagefile):
  image, label = load_image_with_label(imagefile)
  image = resize(image, 150, 150)
  image = normalize(image)
  return image, label

In [13]:
batch_size = 32
shuffle_buffer_size = 1000

In [14]:
train_dataset = train_dataset.map(load_image_train)
train_dataset = train_dataset.shuffle(shuffle_buffer_size)
train_dataset = train_dataset.batch(batch_size)

In [15]:
validation_dataset = validation_dataset.map(load_image_test)
validation_dataset = validation_dataset.batch(batch_size)

In [16]:
base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(150,150,3))

In [17]:
base_model.trainable = False

In [18]:
num_classes = len(class_names)

In [19]:
flatten_layer = tf.keras.layers.GlobalAveragePooling2D()
dense_layer = tf.keras.layers.Dense(32, activation='relu')
dropout_layer = tf.keras.layers.Dropout(0.5)
prediction_layer = tf.keras.layers.Dense(num_classes, activation='softmax')

In [20]:
model = tf.keras.Sequential ([
      base_model,
      flatten_layer,
      dense_layer,
      dropout_layer,
      prediction_layer
])

In [21]:
model.compile(tf.keras.optimizers.RMSprop(learning_rate=0.01), loss=tf.keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [22]:
history = model.fit(train_dataset, epochs=10, validation_data=validation_dataset)

Epoch 1/10
63/63 [==============================] - 678s 11s/step - loss: 0.6710 - accuracy: 0.6155 - val_loss: 0.4776 - val_accuracy: 0.7840
Epoch 2/10
63/63 [==============================] - 676s 11s/step - loss: 0.5170 - accuracy: 0.7300 - val_loss: 0.4716 - val_accuracy: 0.7660
Epoch 3/10
63/63 [==============================] - 677s 11s/step - loss: 0.4964 - accuracy: 0.7495 - val_loss: 0.5161 - val_accuracy: 0.7270
Epoch 4/10
63/63 [==============================] - 677s 11s/step - loss: 0.4554 - accuracy: 0.7820 - val_loss: 0.4227 - val_accuracy: 0.7840
Epoch 5/10
63/63 [==============================] - 674s 11s/step - loss: 0.4732 - accuracy: 0.7780 - val_loss: 0.4566 - val_accuracy: 0.7760
Epoch 6/10
63/63 [==============================] - 696s 11s/step - loss: 0.4138 - accuracy: 0.8020 - val_loss: 1.2611 - val_accuracy: 0.5930
Epoch 7/10
63/63 [==============================] - 676s 11s/step - loss: 0.4289 - accuracy: 0.7870 - val_loss: 0.4226 - val_accuracy: 0.8160
Epoch 